In [8]:
import pandas as pd
import json

In [13]:
# Replace 'path_to_file.json' with your JSON file's path
with open('/Users/kieranshaw/audit-bill-extraction/tests/blueshield_ca_response.json', 'r') as file:
    startTrigger = json.load(file)

In [22]:
invoice_number = startTrigger["parsed_document"]["invoice_number"]["value"]
group_number = startTrigger["parsed_document"]["account_number"]["value"]
total_current_premium = startTrigger["parsed_document"]["total_current_premium"]["value"]
bill_start_date = startTrigger["parsed_document"]["billing_start_date"]["value"]
billing_end_date = startTrigger["parsed_document"]["billing_end_date"]["value"]

In [61]:
import pandas as pd

# Assuming 'line_items' is your original data extracted from the JSON file
line_items = startTrigger["parsed_document"]["line_items"]

# Step 1: Extracting data from 'line_items'
extracted_data = {}
for column in line_items['columns']:
    column_id = column['id']
    column_values = []
    for value in column['values']:
        if value is not None and 'value' in value:
            column_values.append(value['value'])
        else:
            column_values.append(None)  # Append None if the value is missing
    extracted_data[column_id] = column_values

# Creating the initial DataFrame from the extracted data
df = pd.DataFrame(extracted_data)

# Step 2: Filter out rows where employee_name is "Subscriber Name"
df_filtered = df[df['employee_name'] != "Subscriber Name"]

# Step 3: Transform the DataFrame to have a row for each employee-plan type combination
# Melting the DataFrame to long format
df_long = df_filtered.melt(id_vars=['employee_name', 'subscriber_id'],
                           value_vars=['medical_premium', 'dental_premium', 'vision_premium', 'life_premium'],
                           var_name='plan_type', value_name='rate')

plan_type_mapping = {
    'medical_premium': 'medical',
    'dental_premium': 'dental',
    'vision_premium': 'vision',
    'life_premium': 'life'
}

# Convert 'rate' to a numeric type for proper comparison
df_long['rate'] = pd.to_numeric(df_long['rate'], errors='coerce')

# Step 4: Remove rows where the rate is 0
df_long = df_long[df_long['rate'] != 0]

df_long['plan_type'] = df_long['plan_type'].map(plan_type_mapping)
df_long['status'] = 'Active'
df_long['subscriber_id'] = df_long['subscriber_id'].astype(int)
